In [2]:
import pandas as pd
import numpy as np
from pandas import datetime
from matplotlib import pyplot as plt

<ipython-input-2-0c95b2fc7292>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [3]:
def parser(x):
    return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

input_file = './data/AirQualityUCI_refined.csv'

df = pd.read_csv(input_file,
                 index_col=[0],
                 parse_dates=[0],
                 date_parser=parser)

In [5]:
# Visualization setup
%matplotlib
from matplotlib import pyplot as plt
import seaborn; seaborn.set() # set plot styles
%config InlineBackend.figure_format = 'svg'
plt.rcParams['figure.figsize'] = [10, 5]
plt.ion() #enable the interactive mode

Using matplotlib backend: Qt5Agg


In [6]:
# Visualize the 'CO(GT)' variable
df['CO(GT)'].plot()

In [7]:
# Linear interpolation
co = df['CO(GT)'].copy()
co.interpolate(inplace=True) # == co.interpolate(inplace=True)

In [8]:
# Visualize original and imputed data
plt.plot(df['CO(GT)'], label = 'original', zorder = 2)
plt.plot(co, label = 'linear interpolation', zorder = 1)
plt.legend(loc='best')

In [9]:
# Detecting outliers using Boxplot
plt.boxplot(co)
plt.title("Detecting outliers using Boxplot")
plt.xlabel('CO(GT)')

Text(0.5, 0, 'CO(GT)')

In [10]:
# Calculate correlations between variables
corr_matrix = df.corr()
print(corr_matrix)

                 CO(GT)  PT08.S1(CO)  PT08.S2(NMHC)   NOx(GT)  PT08.S3(NOx)  \
CO(GT)         1.000000     0.877203       0.914973  0.792557     -0.701703   
PT08.S1(CO)    0.877203     1.000000       0.892964  0.713654     -0.771938   
PT08.S2(NMHC)  0.914973     0.892964       1.000000  0.704435     -0.796703   
NOx(GT)        0.792557     0.713654       0.704435  1.000000     -0.655707   
PT08.S3(NOx)  -0.701703    -0.771938      -0.796703 -0.655707      1.000000   
NO2(GT)        0.679262     0.641529       0.646245  0.763111     -0.652083   
PT08.S4(NO2)   0.639470     0.682881       0.777254  0.233731     -0.538468   
PT08.S5(O3)    0.851403     0.899324       0.880578  0.787046     -0.796569   
RH             0.040218     0.114606      -0.090380  0.221032     -0.056740   
AH             0.065809     0.135324       0.186933 -0.149323     -0.232017   
C6H6(GT)       0.845144     0.883795       0.981950  0.626638     -0.735744   

                NO2(GT)  PT08.S4(NO2)  PT08.S5(O3) 

In [12]:
# Choose the least correlated variable
rh = df['RH'].copy().interpolate() # RH(Relative Humidity)

In [13]:
# Visualize a scatter plot(CO, RH)
plt.scatter(co, rh, s=12, c='black')
plt.xlabel('CO(GT)')
plt.ylabel('RH')

Text(0, 0.5, 'RH')

In [14]:
# Choose the most correlated variable
nmhc = df['PT08.S2(NMHC)'].copy().interpolate()

In [15]:
# Visualize a scatter plot(CO, NMHC)
plt.scatter(co, nmhc, s=12, c='black')
plt.xlabel('CO(GT)')
plt.ylabel('PT08.S2(NMHC)')

Text(0, 0.5, 'PT08.S2(NMHC)')

In [17]:
"""
IQR-based Outlier Detection
"""

# Q1, Q2(median), Q3
q1 = co.quantile(0.25)# 25% value
median = co.quantile(0.5)    
q3 = co.quantile(0.75)
print(q1, median, q3)

1.1 1.8 2.9


In [18]:
# IQR, upper_fence, lower_fence
iqr = q3 - q1
upper_fence = q3 + 1.5*iqr
lower_fence = q1 - 1.5*iqr
print(iqr, upper_fence, lower_fence)

1.7999999999999998 5.6 -1.5999999999999996


In [20]:
# Filtering the outliers
outliers = co.loc[(co > upper_fence) | (co < lower_fence)]
print(outliers)                

Datetime
2004-03-11 19:00:00    6.9
2004-03-11 20:00:00    6.1
2004-03-12 20:00:00    6.6
2004-03-14 20:00:00    5.9
2004-03-15 09:00:00    8.1
                      ... 
2005-03-23 19:00:00    6.2
2005-03-23 20:00:00    7.2
2005-03-24 19:00:00    5.9
2005-03-24 20:00:00    7.5
2005-03-25 19:00:00    5.7
Name: CO(GT), Length: 224, dtype: float64


In [22]:
# Mask for outliers
mask = co.index.isin(outliers.index)
mask

array([False, False, False, ..., False, False, False])

In [23]:
# Visualize the normal data and outliers
plt.plot(co[~mask], label = 'normal', color='blue',
        marker = 'o', markersize=3, linestyle = 'None')
plt.plot(outliers, label = 'outliers', color = 'red',
        marker = 'x', markersize = 3, linestyle = 'None')
plt.legend(loc='best')

In [25]:
# Removing the outliers
co_refind = co.copy()
co_refind[mask] = np.nan
print(co_refind)
co_refind.plot()

Datetime
2004-03-10 18:00:00    2.6
2004-03-10 19:00:00    2.0
2004-03-10 20:00:00    2.2
2004-03-10 21:00:00    2.2
2004-03-10 22:00:00    1.6
                      ... 
2005-04-04 10:00:00    3.1
2005-04-04 11:00:00    2.4
2005-04-04 12:00:00    2.4
2005-04-04 13:00:00    2.1
2005-04-04 14:00:00    2.2
Name: CO(GT), Length: 9357, dtype: float64


In [27]:
"""
Detecting Outliers with Z-Scores
"""

# Visualize the distribution of the 'CO(GT)' variable
import seaborn as sns
sns.distplot(co)

In [28]:
# Mean, Standard deviation
mean = np.mean(co)
std = np.std(co)
print(mean, std)

2.126146200705357 1.4369815748596482


In [29]:
outliers = []
thres = 3 # z-Score threshold

for i in co:
    z_score = (i-mean) / std
    if(np.abs(z_score) > thres):
        print(z_score)
        outliers.append(i)

3.3221398818289734
3.113368937755246
4.157223658123882
4.087633343432639
3.0437786230640036
3.113368937755246
3.8092720846676698
3.182959252446488
3.113368937755246
3.600501140593942
3.5309108259027
3.0437786230640036
3.0437786230640036
3.182959252446488
3.0437786230640036
3.600501140593942
3.182959252446488
3.7396817699764275
3.0437786230640036
3.182959252446488
3.3917301965202156
4.087633343432639
3.878862399358912
3.2525495671377302
4.36599460219761
3.2525495671377302
4.018043028741397
4.36599460219761
4.087633343432639
4.36599460219761
5.131488063801275
3.0437786230640036
3.113368937755246
3.7396817699764275
3.2525495671377302
4.018043028741397
4.087633343432639
4.922717119727547
3.3917301965202156
3.786075313103922
4.180420429687629
4.574765546271336
4.157223658123882
4.087633343432639
3.113368937755246
3.2525495671377302
3.9484527140501537
5.618620266639971
3.113368937755246
3.7396817699764275
3.5309108259027
4.296404287506367
4.505175231580093
3.3221398818289734
3.04377862306400

In [30]:
# Simplified version of filltering outliers
outliers = co.loc[np.abs((co - mean)/std > 3)].copy()

In [34]:
# Mask for outliers
mask = co.index.isin(outliers.index)
mask

array([False, False, False, ..., False, False, False])

In [36]:
# Comparison of distributions before/after outlier removal
sns.distplot(co, axlabel='CO(GT)', label='original')
sns.distplot(co[~mask], label='outliers removed')
plt.legend(loc='best')

In [38]:
# [exer] Adjust thres

# Flooring and Capping
floor = co.quantile(0.1)
cap = co.quantile(0.9)

co.loc[co < floor] = floor
co.loc[co > cap] = cap

# Visualize the result
co.plot()